In [ ]:
!pip install -q faster-whisper
!pip install langchain_community
!pip install -U transformers langchain faiss-cpu bitsandbytes accelerate
!pip install git+https://github.com/huggingface/parler-tts.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 18.8

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install pydub
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from pydub import AudioSegment
import tempfile
import os
import json
import re

def extract_final_answer_json(llm_output: str) -> str:
    """
    Extract the final valid JSON object from LLM output that contains only the 'answer' key.
    Returns a clean string containing just the answer value, or a message if not found.
    """

    # Find all potential JSON blocks using regex
    json_blocks = re.findall(r'{\s*"answer"\s*:\s*"(.+?)"\s*}', llm_output, re.DOTALL)

    if not json_blocks:
        return "❌ No valid answer JSON found."

    # The last one is most likely the final real answer (not placeholder)
    last_answer = json_blocks[-1].strip()

    # Optional: Check if it's a placeholder
    placeholders = [
        "Your helpful answer here",
        "Only include the final resolution steps",
        "No relevant data",
    ]
    if any(ph.lower() in last_answer.lower() for ph in placeholders):
        return "❌ Found placeholder, not a real answer."

    return last_answer



def split_hindi_sentences(text):
    import re
    return [s.strip() for s in re.split(r"[।.]\s*", text) if s.strip()]

def generate_single_audio_file(text, description, output_path="final_output.wav"):
    sentences = split_hindi_sentences(text)
    final_audio = AudioSegment.empty()

    for i, sentence in enumerate(sentences):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmpfile:
            tmp_path = tmpfile.name

        # Generate individual sentence audio
        text_to_speech_parler(
            sentence,
            description=description,
            output_path=tmp_path
        )

        # Load and append to final audio
        audio_segment = AudioSegment.from_wav(tmp_path)
        final_audio += audio_segment
        os.remove(tmp_path)

    # Export merged final audio
    final_audio.export(output_path, format="wav")
    print(f"✅ Final audio saved to: {output_path}")


In [ ]:
import sys
import unicodedata
import torch
sys.path.append("/content/drive/MyDrive/LG_project/modules")

from transcriber import transcribe_audio_whisper
from mistral_loader import load_mistral
from local_mistral_llm import LocalMistralLLM
from vector_search import load_vector_index
from parler_pipeline import translate, text_to_speech_parler
from langchain.chains import RetrievalQA
from hybrid_response import hybrid_answer
# print("Loading models and index once")
# mistral_path = "/content/drive/MyDrive/LG_project/mistral_local/mistral_local"
# index_path = "/content/drive/MyDrive/LG_project/pdf_index"

# model, tokenizer = load_mistral(mistral_path)
# print(" Model loaded")

# print(f" Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
# print(f" Reserved : {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

# llm = LocalMistralLLM(model=model, tokenizer=tokenizer)
# vectorstore = load_vector_index(index_path)
# retriever = vectorstore.as_retriever()
# qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
LANG_CODE_MAP = {
    "hi": "hi_IN",
    "ta": "ta_IN",
    "te": "te_IN",
    "bn": "bn_IN",
    "pa": "pa_IN",
    "gu": "gu_IN",
    "kn": "kn_IN",
    "ml": "ml_IN",
    "mr": "mr_IN",
    "or": "or_IN",
    "default": "en"
}

def clean_mixed_to_hindi(text: str) -> str:
    replacements = {
        "ولتاژ": "वोल्टेज",
        "ریفریجریٹ": "रेफ्रिजरेंट",
        "درجہ حرارت": "तापमान",

        "MCB": "एमसीबी",
        "refrigerant": "रेफ्रिजरेंट",
        "specifications": "विवरण",
        "filter": "फिल्टर",
        "mode": "मोड",
        "price": "कीमत",
        "model": "मॉडल",
        "unit": "यूनिट",
        "AC": "एसी",
    }

    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)

    return text


def get_translation_lang_code(whisper_lang):
    if whisper_lang == "en":
        return None
    return LANG_CODE_MAP.get(whisper_lang, LANG_CODE_MAP["default"])

def process_audio(audio_path, output_path="final_response.wav"):
    print(f"\n Processing: {audio_path}")

    whisper_lang, user_text = transcribe_audio_whisper(audio_path)
    print(f" Transcribed: {user_text}")
    print(f" Detected language: {whisper_lang}")

    # response = qa_chain.run(user_text).strip()
    response=hybrid_answer(user_text)
    print(response)

    # print(f" Response from LLM: {response}")
    response=extract_final_answer_json(response)
    print(response)
    target_lang_code = get_translation_lang_code(whisper_lang)
    print(f" Target language code: {target_lang_code}")
    if target_lang_code == "en":
        translated = response

    else:
        translated = translate(response, target_lang_code)
        print(f" Translated to {target_lang_code}: {translated}")

    text = unicodedata.normalize("NFC", translated)
    text=clean_mixed_to_hindi(text)
    print(f" Cleaned text: {text}")

    description="""A male speaker from North India (Delhi/UP region) speaking fluently with a clear, natural accent and slightly slow, easy-to-understand pronunciation. Use commonly spoken Hindi words and natural intonation."""
    generate_single_audio_file(text,description,"response.wav")
    print(f" TTS saved to: {output_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
process_audio("/content/Recording (5).m4a")


 Processing: /content/Recording (5).m4a
 Transcribed: How to fix AC cooling
 Detected language: hi


/content/drive/MyDrive/LG_project/modules/hybrid_response.py:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_question)
/content/drive/MyDrive/LG_project/modules/hybrid_response.py:59: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_response = llm(prompt)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


❌ JSON parse error: Expecting value: line 2 column 1 (char 1)
Raw output: 
You are an assistant that answers customer queries using ONLY the provided information.

---

USER QUESTION:
How to fix AC cooling

---

DATABASE INFO:
No relevant data.

---

DOCUMENT INFO:
Steps to fix:  
1. Check the MCB; it must be ON.  
2. Check the voltage (should be 200V–240V).

---

2. Q: How To Fix No Cooling Issue?  
A: Low cooling may be caused by:  
- Incorrect mode setting  
- Dirty or blocked filters  
- Low voltage (below 200V)  

Steps to fix:  
1. Select the Cooling Mode.  
2. Set the temperature to 18°C.  
3. Set the fan speed to High.  
4. Clean dusty air filters.  
5. Ensure voltage is 200V–240V.

---

3. Q: How To Clean The AC Filter?  
A: Cleaning improves cooling and efficiency.  

Steps:  
1. Lift and remove the air filter from the indoor unit.  
2. Clean with water and brush.  
3. Let it dry completely.  
4. Re-install the dry filter into the indoor unit.  

*Note: Clean every 15 days.  

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Translated to hi_IN: AC की वोल्टेज जाँचें (200V-240V होना चाहिए)। यदि वोल्टेज कम है तो एक पेशेवर से परामर्श करें। यदि वोल्टेज सही है तो हवा फिल्टरों को साफ करने का प्रयास करें। गलत मोड सेटिंग, दूषित फिल्टर और खराब वाष्पीकरण भी कम शीतलता पैदा कर सकते हैं। फिल्टरों को साफ करने के लिए, पानी और ब्रश से हटाएं और साफ करें, फिर पूरी तरह से सूख दें और पुनः स्थापित करें। तापमान सेटिंग का समायोजन करें और सही वाष्पीकरण और अंतराल बनाए रखें ताकि शीतलता में सुधार किया जा सके।
 Cleaned text: एसी की वोल्टेज जाँचें (200V-240V होना चाहिए)। यदि वोल्टेज कम है तो एक पेशेवर से परामर्श करें। यदि वोल्टेज सही है तो हवा फिल्टरों को साफ करने का प्रयास करें। गलत मोड सेटिंग, दूषित फिल्टर और खराब वाष्पीकरण भी कम शीतलता पैदा कर सकते हैं। फिल्टरों को साफ करने के लिए, पानी और ब्रश से हटाएं और साफ करें, फिर पूरी तरह से सूख दें और पुनः स्थापित करें। तापमान सेटिंग का समायोजन करें और सही वाष्पीकरण और अंतराल बनाए रखें ताकि शीतलता में सुधार किया जा सके।
🎯 Device Check:
desc_enc[input_ids]: cuda:0
desc_enc[attention_mask]: 